## Raspberry Pi Zero W PDM Mode Setup

<b> RPi type: Zero W </b>

<b> RPi version: 4.19.88+ </b>

The RPi's SoC (BCM2835) is not configured to take PDM data as an input on its I2S line, however it is capable of interfacing with 2 PDM microphones. To enable PDM input, these are the specific steps that I followed:

Navigate to the BCM2835 sound source file (bcm2835-i2s.c) location by doing the following:
 
 \$ cd linux/sound/soc/bcm

In [4]:
from IPython.display import display, HTML
display(HTML('''<img src="Images/PDM_Setup_1.PNG" width="800" />'''))

Open the BCM2835 I2S sound source file for editing in root mode

\$ sudo nano bcm2835-i2s.c

In [7]:
display(HTML('''<img src="Images/PDM_Setup_2.PNG" width="800" />'''))

In [ ]:
   - Go to line ~393/394 and write bclk_rate = 3072000 (This fixes the PCM clock rate at 3.072 MHz which is what we need for PDM microphones)
     - Go to line ~590 and write "mode |= BCM2835_I2S_PDMN;" on one line, and "mode |= BCM2835_I2S_PDME" on the next line, <b> before the "regmap_write(dev->i2s_regmap, BCM2835_I2S_MODE_A_REG, mode); </b> (This ensures PDM mode is enabled and a decimation factor of N = 32 is selected during the I2S mode setup).
 - \\$ make -C /lib/modules/(\\$ uname -r )/build M=(\$ pwd) modules (Now that we have changed the source file, we need to use the 'make' command to run the Makefile which will compile the changes and generate a .ko kernel object file which is what we want - this file is called "snd-soc-bcm2835-i2s.ko") 
 - \$ cd /lib/modules/(uname -r)/kernel/sound/soc/bcm (This puts us in the directory of the kernel which contains the existing "snd-soc-bcm2835-i2s.ko" which needs to be replaced)
     - \$ sudo rm -r snd-soc-bcm2835-i2s.ko (This removes the existing kernel object so we can replace it with out newly generated .ko file)
 - In the /linux/sound/soc/bcm directory, execute the command: sudo cp snd-soc-bcm2835-i2s.ko /lib/modules/([dollar sign] uname -r)/kernel/sound/soc/bcm (This will copy over the newly generated .ko file with PDM mode enabled to where we need it!)
 - Reboot the RPi using \\$ sudo reboot and use the following command to record audio: arecord -D plughw:1 -c1 -r 48000 -f S16_LE -d 60 -t wav -V mono -v test.wav

Make the following edits in the sound source file to ensure PDM is enabled correctly - here we are using bitwise OR to write 1 to the relevant PDM registers which you can find in the BCM2835 SOC datasheet:
 - Go to line ~393/394 and write "bclk_rate = 307200" (This fixes the PCM clockrate at 3.072 MHz which is what we generally need for PDM microphones - note this may be unique for different microphones, check your relevant datasheet)
 - Go to line ~586 and write "mode |= BCM2835_I2S_PDMN;" on one line and "mode |= BCM2835_I2S_PDME" on the next line. Note this should be done before the existing command in the file "regmap_write(dev->i2s_regmap, BCM2835_I2S_MODE_A_REG, mode);" (This enables PDM mode and selects a decimation factor of N = 32)
 
Be sure to save the changes made to the sound source file.  

In [9]:
display(HTML('''<img src="Images/PDM_Setup_3.PNG" width="800" />'''))
display(HTML('''<img src="Images/PDM_Setup_4.PNG" width="800" />'''))

Now that we have changed the source file, we use the 'make' command to run the MakeFile which compiles all changes and generates a .ko kernel object:
    
\\$ make -C /lib/modules/(\\$ uname -r)/build M=(\$ pwd)

You should see a bunch of files with .c, .o and .ko extensions - look out for the file named <b> "snd-soc-bcm2835-i2s.ko" </b>. This is the file which captures changes made in the bcm2835-i2s.c file.

In [11]:
display(HTML('''<img src="Images/PDM_Setup_5.PNG" width="800" />'''))

Now we have to navigate to the directory of the kernel which contains the existing "snd-soc-bcm2835-i2s.ko" which needs to be replaced. Once we are there, we need to remove the existing kernel object so we can replace it with the newly generated one:

\$ cd lib/modules/(uname -r)/kernel/sound/soc/bcm

\$ sudo rm -r snd-soc-bcm2835-i2s.ko

In [14]:
display(HTML('''<img src="Images/PDM_Setup_6.PNG" length="600" />'''))

Now we navigate back to the location of our <b> new </b> "snd-soc-bcm2835-i2s.ko" kernel object and copy this over to the kernel directory in which we have just deleted the existing sound kernel object:
    
sudo cp snd-soc-bcm2835-i2s.ko /lib/modules/([dollar sign] uname -r)/kernel/sound/soc/bcm